<a href="https://colab.research.google.com/github/kes1492/Korea_NLP_KimEuiSoon/blob/main/%EC%98%81%EC%96%B4%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_Friends_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

# 영어 감성분석 (BERT)#

### Friends data set 

#####   출처 : 아래 제작자 기본모델 사용
#####   수업용 자료 



##### **Settings**

In [1]:
!pip install transformers --quiet # package installer for python


     |████████████████████████████████| 1.5MB 9.3MB/s 
     |████████████████████████████████| 2.9MB 30.8MB/s 
     |████████████████████████████████| 890kB 41.1MB/s 


In [2]:
import torch
from transformers import BertModel, BertTokenizer

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Dec 25 06:38:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

##### **Tokenization**

In [5]:
sentence = 'All the classes are provided.'

In [6]:
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)

['[CLS]', 'all', 'the', 'classes', 'are', 'provided', '.', '[SEP]']


In [7]:
ids = [tokenizer.convert_tokens_to_ids(tokens)]
print(ids)

[[101, 2035, 1996, 4280, 2024, 3024, 1012, 102]]


In [8]:
input_tensor = torch.tensor(ids)
print(input_tensor.data)

tensor([[ 101, 2035, 1996, 4280, 2024, 3024, 1012,  102]])


##### **Model**

In [9]:
hidden_tensor = model(input_tensor)[0]
print(hidden_tensor.size())

torch.Size([1, 8, 768])


In [10]:
logit = torch.nn.Linear(768, 2)(hidden_tensor)
print(logit.size())
print(logit.data)

torch.Size([1, 8, 2])
tensor([[[ 0.1387,  0.1490],
         [-0.0563,  0.0236],
         [ 0.1951, -0.0306],
         [-0.0608, -0.1247],
         [ 0.0069, -0.4530],
         [ 0.1424, -0.1341],
         [-0.2533,  0.5812],
         [-0.1154,  0.4176]]])


In [11]:
prediction = torch.nn.Softmax(dim=-1)(logit)
print(prediction.size())
print(prediction.data)

torch.Size([1, 8, 2])
tensor([[[0.4974, 0.5026],
         [0.4800, 0.5200],
         [0.5562, 0.4438],
         [0.5160, 0.4840],
         [0.6130, 0.3870],
         [0.5687, 0.4313],
         [0.3027, 0.6973],
         [0.3698, 0.6302]]])


# **Emotion Recognition**

##### **Dataset**

In [12]:
#import re
#import urllib.request

#urllib.request.urlretrieve("https://raw.githubusercontent.com/kes1492/Korea_NLP_KimEuiSoon/main/friends_dev.jason", filename="friends_dev.jason")

!git clone https://github.com/kes1492/Korea_KimEuiSoon_Data.git
!cp ./Korea_KimEuiSoon_Data/* .

Cloning into 'Korea_KimEuiSoon_Data'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [13]:
import json

data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
  for dialog in json.loads(open('friends_' + dtype + '.json').read()):
    for line in dialog:
      data[dtype]['speaker'].append(line['speaker'])
      data[dtype]['utterance'].append(line['utterance'])
      data[dtype]['emotion'].append(line['emotion'])

print(data)

{'train': {'speaker': ['Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Joey', 'Rachel', 'Joey', 'Rachel', 'Joey', 'Joey', 'Joey', 'Joey', 'Rachel', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Monica', 'Chandler', 'Phoebe', 'Chandler', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Chandler', 'Phoebe', 'Joey', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Ross', 'Rachel', 'Phoebe', 'Sergei', 'Phoebe', 'Sergei', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Chandler', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Joey', 'Ross', 'Phoebe', 'Phoeb

In [14]:
e2i_dict = dict((emo, i) for i, emo in enumerate(set(data['train']['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}

##### **Model**

In [15]:
import torch.nn as nn
from transformers import BertModel, BertTokenizer
# Fine tuning을 위한 config 정의
#from argparse import Namespace 
#configs = Namespace()

#configs.per_gpu_eval_batch_size = 32

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
    self.bert_model = BertModel.from_pretrained(pretrained_weights)
    self.linear = torch.nn.Linear(768, len(e2i_dict))

  def forward(self, utterance):
    tokens = self.bert_tokenizer.tokenize(utterance)
    tokens = ['[CLS]'] + tokens + ['[SEP]'] # (len)
    ids = [tokenizer.convert_tokens_to_ids(tokens)] # (bat=1, len)
    input_tensor = torch.tensor(ids).cuda()

    hidden_tensor = self.bert_model(input_tensor)[0] # (bat, len, hid)
    hidden_tensor = hidden_tensor[:, 0, :] # (bat, hid)
    logit = self.linear(hidden_tensor)
    return logit

##### **Evaluation Metrics**

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(true_list, pred_list):
  precision = precision_score(true_list, pred_list, average=None)
  recall = recall_score(true_list, pred_list, average=None)
  micro_f1 = f1_score(true_list, pred_list, average='micro')
  print('precision:\t', ['%.4f' % v for v in precision])
  print('recall:\t\t', ['%.4f' % v for v in recall])
  print('micro_f1: %.6f' % micro_f1)

##### **Hyper-parameters**

In [17]:
pretrained_weights = 'bert-base-uncased'
#pretrained_weights = 'bert-base-multilingual-cased'
learning_rate = 1e-5
n_epoch = 15 # 3-> 15

##### **Training**

In [18]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from tqdm import tqdm_notebook

model = Model()
model.cuda()
criterion = torch.nn.CrossEntropyLoss() # LogSoftmax & NLLLoss
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for i_epoch in range(n_epoch):
  print('i_epoch:', i_epoch)

  model.train()
  for i_batch in tqdm_notebook(range(len(data['train']['utterance']))):
    logit = model(data['train']['utterance'][i_batch])
    target = torch.tensor([e2i_dict[data['train']['emotion'][i_batch]]]).cuda()
    loss = criterion(logit, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  
  model.eval()
  pred_list, true_list = [], []
  for i_batch in tqdm_notebook(range(len(data['dev']['utterance']))):
    logit = model(data['dev']['utterance'][i_batch])
    _, max_idx = torch.max(logit, dim=-1)
    pred_list += max_idx.tolist()
    true_list += [e2i_dict[data['dev']['emotion'][i_batch]]]
  evaluate(pred_list, true_list) # print results



i_epoch: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



precision:	 ['0.0000', '0.1304', '0.6341', '0.2903', '0.3647', '0.5099', '0.1355', '0.9165']
recall:		 ['0.0000', '0.4286', '0.5693', '0.5455', '0.3407', '0.6364', '0.3372', '0.6401']
micro_f1: 0.582343
i_epoch: 1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...


precision:	 ['0.1724', '0.1739', '0.6260', '0.2903', '0.3647', '0.5629', '0.3879', '0.7536']
recall:		 ['0.5000', '0.4444', '0.5461', '0.5000', '0.4429', '0.5592', '0.3294', '0.7283']
micro_f1: 0.571307
i_epoch: 5



precision:	 ['0.0690', '0.2609', '0.4309', '0.3871', '0.3176', '0.6291', '0.3224', '0.7373']
recall:		 ['0.3333', '0.1875', '0.5824', '0.4800', '0.4219', '0.5108', '0.2887', '0.7098']
micro_f1: 0.541596
i_epoch: 6



precision:	 ['0.0690', '0.3043', '0.5285', '0.2742', '0.4000', '0.5894', '0.2897', '0.7393']
recall:		 ['0.3333', '0.2333', '0.5909', '0.5484', '0.4857', '0.4684', '0.2831', '0.6954']
micro_f1: 0.542445
i_epoch: 7



precision:	 ['0.0690', '0.3043', '0.5447', '0.2903', '0.2941', '0.6159', '0.3131', '0.7251']
recall:		 ['0.2500', '0.3500', '0.5447', '0.4186', '0.4630', '0.4559', '0.3032', '0.7050']
micro_f1: 0.539049
i_epoch: 8



precision:	 ['0.0690', '0.4348', '0.6098', '0.3387', '0.3294', '0.5430', '0.3411', '0.6415']
recall:		 ['0.2222', '0.2128', '0.5357', '0.3621', '0.3944', '0.5290', '0.2674', '0.7412']
micro_f1: 0.514431
i_epoch: 9


In [ ]:


# CPU로 데이터 이동
logit = logit.detach().cpu().numpy()
label_ids = b_labels.to('cpu').numpy()

# 정확도 계싼
tmp_eval_accuracy = flat_accuracy(logits, label_ids)
eval_accuracy += tmp_eval_accuracy
nb_eval_steps += 1

print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))


##### **Labeling**


In [ ]:
from collections import OrderedDict
labeled = []
dialogs = json.loads(open('unlabeled.json').read())
for dialog in tqdm_notebook(dialogs):
  dialog_list = []
  for line in dialog:
    logit = model(line['utterance'])
    _, max_idx = torch.max(logit, dim=-1)
    pred_emotion = max_idx.tolist()[0]

    line_dict = OrderedDict()
    line_dict['speaker'] = line['speaker']
    line_dict['utterance'] = line['utterance']
    line_dict['emotion'] = i2e_dict[pred_emotion]
    dialog_list.append(line_dict)
  labeled.append(dialog_list)

with open('labeled.json', 'w') as json_file:
  json.dump(labeled, json_file, indent='\t')

# Test Data 로딩 및 예측

In [ ]:
#########################################################################
# Open Friends dataset from csv file
import pandas as pd
import numpy as np

en_data = pd.read_csv("en_data.csv", sep=',')

In [ ]:
# Load dataset 
en_data_doc = en_data['utterance']
index = 0
predict_value = [] #예측값 저장 리스트

model.eval()
for sentence in en_data_doc:
  temp_doc = []
  value = 0

  temp_doc = en_data_doc[index]
  logit = model(temp_doc)
  _, max_idx = torch.max(logit, dim=-1)
  pred_emotion = max_idx.tolist()[0]
  predict_value.append(i2e_dict[pred_emotion])
  index = index + 1


In [ ]:
id_value = [] #id 저장 리스트

#id 예측 list만큼 0부터 순차적으로 저장
id_index = 0
for i in range(len(en_data_doc)):
  id_value.append(id_index)
  id_index = id_index +1

en_data_doc_out = pd.DataFrame({'Id':pd.Series(id_value), 'Predicted':pd.Series(predict_value)})

df = pd.DataFrame(en_data_doc_out)
df.to_csv('en_predicted(2).csv', index=False)